# Notebook for getting nodule_probability_vector for each region in Kaggle-Dataset image

This notebook takes data from Kaggle-data and creates probabilty of nodule presence in each region of the image

In [1]:
import pydicom # for reading dicom files
from scipy import ndimage as nd
import os # for doing directory operations 
import pandas as pd # for some simple data analysis (right now, just to load in the labels data and quickly reference it)
import numpy as np
import sklearn

data_dir = 'D:\\riya\\DL-MP\\kaggle\\stage1\\stage1\\'
patients = os.listdir(data_dir)
labels_df = pd.read_csv('D:\\riya\\DL-MP\\kaggle\\stage1_labels\\stage1_labels.csv', index_col=0)
print(len(patients))
labels_df.head(50)

1397


,cancer
id,
0015ceb851d7251b8f399e39779d1e7d,1
0030a160d58723ff36d73f41b170ec21,0
003f41c78e6acfa92430a057ac0b306e,0
006b96310a37b36cccb2ab48d10b49a3,1
008464bb8521d09a42985dd8add3d0d2,1
0092c13f9e00a3717fdc940641f00015,0
00986bebc45e12038ef0ce3e9962b51a,0
00cba091fa4ad62cc3200a657aeb957e,0
00edff4f51a893d80dae2d42a7f45ad1,1


In [2]:
def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]


In [ ]:
import matplotlib.pyplot as plt
from keras.models import load_model
import cv2

for patient in patients:
    print(patient)
    #label = labels_df.get_value(patient, 'cancer')
    label = labels_df.at[patient, 'cancer']
    path = data_dir + patient
    
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    #cv2.imshow('image',slices[0])
    num = len(slices)/36
    num_int = int(num)
    last_slice = 36*num_int
    
    #print(slices)
    #slices = np.array(slices[:last_slice])
    
    temp_arr = [crop_center(slices[i].pixel_array, 504, 504) for i in range(len(slices))]
    temp_arr = np.array(temp_arr)
    print(temp_arr.shape)
    zoomFactors = [504/float(a) for a in temp_arr.shape]

    temp_arr = nd.interpolation.zoom(temp_arr, zoom=zoomFactors) 
    print(temp_arr.shape)
    new_arr =  np.split(temp_arr, 14)
    
    
    
    all_splits = []
    
    model = load_model('C:\\Users\\dc\\Documents\\riya\\Lung-Cancer-Detection-master\\Output\\model.h5')

    all_elems = []
    for item in new_arr:

        
        nd_arr = np.split(item,14,1)
        nd_arr = np.array(nd_arr)
        
        for it in nd_arr:
            
            nd_new = np.split(it,14, 2)
            nd_new = np.array(nd_new)
            for elem in nd_new:
                
                elem = elem.reshape(1,36,36,36)
                all_elems.append(elem)
    all_elems = np.array(all_elems)
    new_tuple = model.predict_proba(all_elems, batch_size = 32)
    print(type(new_tuple[0][1]))
    print(new_tuple)
    print(len(new_tuple))


Using TensorFlow backend.


0015ceb851d7251b8f399e39779d1e7d
(195, 504, 504)
(504, 504, 504)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
0030a160d58723ff36d73f41b170ec21
(265, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
003f41c78e6acfa92430a057ac0b306e
(233, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0.47609058 0.5239094 ]
 [0.         1.        ]
 [0.         1.        ]
 ...
 [0.         1.        ]
 [0.         1.        ]
 [0.05807708 0.9419229 ]]
2744
006b96310a37b36cccb2ab48d10b49a3
(173, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0.47609058 0.5239094 ]
 [0.47609058 0.5239094 ]
 [0.47609058 0.5239094 ]
 ...
 [0.47609058 0.5239094 ]
 [0.47609058 

C:\Users\dc\Anaconda3\envs\mrudang\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
01e349d34c06410e1da273add27be25c
(159, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[1.1261095e-26 1.0000000e+00]
 [5.3687363e-28 1.0000000e+00]
 [6.8849777e-27 1.0000000e+00]
 ...
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]]
2744
01f1140c8e951e2a921b61c9a7e782c2
(241, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
024efb7a1e67dc820eb61cbdaa090166
(175, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[4.7609058e-01 5.2390939e-01]
 [4.7609058e-01 5.2390939e-01]
 [2.1533521e-36 1.0000000e+00]
 ...
 [4.7609058e-01 5.2390939e-01]
 [4.7609058e-01 5.2390939e-01]
 [4.7609058e-01 5.2390939e-01]]
2744
0257df465d9e4150adef13303433ff1e
(186, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
0268f3a7a174121

(175, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
09fdf599084b816247ba38d95b3c9d80
(120, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
0a099f2549429d29b32f349e95fb2244
(128, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
0a0c32c9e08cc2ea76a71649de56be6d
(133, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0.47609058 0.5239094 ]
 [0.47609058 0.5239094 ]
 [0.47609058 0.5239094 ]
 ...
 [0.47609058 0.5239094 ]
 [0.47609058 0.5239094 ]
 [0.47609058 0.5239094 ]]
2744
0a38e7597ca26f9374f8ea2770ba870d
(110, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
0acbebb8d463b4b9ca88cf38431aac69
(203, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[9.257331e-27 1.000000e+00]
 [4.808481e-30 1.000000e+00]
 [6.650801e-30 1.000

(169, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[1.7577752e-37 1.0000000e+00]
 [6.5654207e-38 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 ...
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]]
2744
12e0e2036f61c8a52ee4471bf813c36a
(134, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
1344e604281e68ac71f25c02e7637992
(352, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
135f21a84ac903a1d8dfbb0dfe46d792
(243, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
13bb12b3b27d5a7b4b142503a1ae9e73
(172, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[8.2770292e-36 1.0000000e+00]
 [2.2896172e-37 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 ...
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]]
2744

(139, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
1acbe17dc8f9f59d2fd167b2aa6c650f
(179, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[4.2600460e-37 1.0000000e+00]
 [3.0636813e-38 1.0000000e+00]
 [5.7315316e-38 1.0000000e+00]
 ...
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]]
2744
1b17c83c3b0cb973e41f9121e562d030
(162, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[6.1140425e-05 9.9993885e-01]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 ...
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]]
2744
1b2355e6054759f2f0eb2542ed382b78
(155, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
1b7ca8dad5c36feb0a6abf8079173e22
(298, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744

(215, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
252b5fd5c4a5eb77be4fbe41df49bd07
(324, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[7.6662479e-37 1.0000000e+00]
 [3.1958792e-36 1.0000000e+00]
 [1.5300572e-36 1.0000000e+00]
 ...
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]]
2744
25782c40c29f7d774f8b644f3e68b590
(197, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
25a3b616f6cc89e74f6502d65b687972
(173, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0.47609058 0.5239094 ]
 [0.47609058 0.5239094 ]
 [0.47609058 0.5239094 ]
 ...
 [0.47609058 0.5239094 ]
 [0.47609058 0.5239094 ]
 [0.47609058 0.5239094 ]]
2744
25c1c4f008e8addc4d386cab58815052
(116, 504, 504)
(504, 504, 504)
<class 'numpy.float32'>
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
2744
25c369973827571098bdebf2a15c6ef2
(1